# 06. RAG Parte 2: Embeddings e Vector Stores

Com os textos divididos, precisamos transformá-los em números (vetores) para que possamos fazer busca semântica (por significado, não por palavra-chave exata).

**Objetivos:**
- Entender o que são Embeddings.
- Criar um Vector Store (FAISS) para armazenar os vetores.
- Fazer uma busca de similaridade.

# Explicação Detalhada do Assunto

# 06. RAG Parte 2: Embeddings e Vector Stores

Bem-vindo ao segundo notebook da nossa jornada em Retrieval Augmented Generation (RAG)! Neste notebook, mergulharemos no coração do RAG, explorando como transformar texto bruto em representações numéricas significativas, que possibilitam a busca semântica eficiente. Preparar-se para dar vida aos seus dados!

## Resumo Executivo

Este notebook demonstra o processo essencial de criação de um "banco de dados" de conhecimento vetorial. Partindo de textos divididos em chunks, aprenderemos como usar embeddings para representá-los como vetores e, em seguida, como indexá-los em um Vector Store (FAISS) para realizar buscas de similaridade semântica. O objetivo final é construir uma base sólida para o nosso sistema RAG, permitindo que ele encontre informações relevantes com base no significado, e não apenas em palavras-chave.

## Conceitos Chave

Para entender completamente o que faremos, vamos revisar alguns conceitos importantes:

*   **Embeddings:** Um modelo de embedding converte texto em um vetor de números (uma lista de floats, por exemplo). Esse vetor representa o significado semântico do texto, permitindo que textos com significados semelhantes tenham vetores próximos no espaço vetorial.
*   **Vector Store:** Um banco de dados especializado para armazenar e indexar vetores (embeddings). Ele permite buscas rápidas e eficientes por vetores similares, o que é crucial para encontrar os chunks de texto mais relevantes para uma determinada consulta. FAISS (Facebook AI Similarity Search) é uma biblioteca popular para construir Vector Stores.
*   **Busca de Similaridade:** O processo de encontrar os vetores mais próximos (mais similares) a um determinado vetor de consulta dentro de um Vector Store. Isso nos permite encontrar os chunks de texto que são semanticamente mais relacionados à pergunta do usuário.

## Objetivos de Aprendizado

Ao concluir este notebook, você será capaz de:

*   Transformar documentos de texto em embeddings usando um modelo como o `GoogleGenerativeAIEmbeddings`.
*   Criar um Vector Store (usando FAISS) para indexar seus embeddings.
*   Realizar buscas de similaridade em seu Vector Store para encontrar chunks de texto relevantes.
*   Entender o papel fundamental dos embeddings e Vector Stores na arquitetura RAG.

## Importância no Ecossistema LangChain

Este notebook é crucial porque estabelece a base para a funcionalidade de busca do nosso sistema RAG. A capacidade de encontrar informações relevantes de forma eficiente é o que permite ao LLM gerar respostas informativas e contextualmente apropriadas. Sem embeddings e Vector Stores, estaríamos limitados à busca por palavras-chave, o que frequentemente resulta em resultados irrelevantes ou incompletos. Ao dominar essas técnicas, você estará preparado para construir sistemas RAG poderosos e eficazes com LangChain.

Vamos começar a transformar texto em conhecimento acionável!

---


In [1]:
### INJECTION START ###
import os
from dotenv import load_dotenv
import sys
# Carrega .env do local ou de pastas comuns
for p in ['.', '..', 'scripts', '../scripts']:
    path = os.path.join(p, '.env')
    if os.path.exists(path):
        load_dotenv(path)
        break
if os.getenv('GOOGLE_API_KEY'):
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
### INJECTION END ###

import os
from dotenv import load_dotenv
import sys
# Autenticação automática do script
for p in ['.', '..', 'scripts', '../scripts']:
    path = os.path.join(p, '.env')
    if os.path.exists(path):
        load_dotenv(path)
        break
if os.getenv('GOOGLE_API_KEY'):
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

import os
from dotenv import load_dotenv
load_dotenv()

# !pip install -qU langchain langchain-openai langchain-community faiss-cpu python-dotenv # Script-patched

False

In [2]:
import os
try:
    from google.colab import userdata
except ImportError:
    userdata = None
import getpass

try:
    pass # Script-patched: using env var
except:
    pass # Script-patched: using env var

## 1. Prep: Recriando os Chunks

Vamos repetir rapidamente o passo anterior para ter os dados.

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Carregando docs pequenos para exemplo
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
]

loader = WebBaseLoader(urls)
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


## 2. Embeddings

Modelo de Embedding converte texto em um vetor de números (ex: lista de 1536 floats).

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings()

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.6). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/auth/__init__.py:54: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade your Python version, and then update google-auth.
  warnings.warn(eol_message.format("3.9"), FutureWarning)
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/oauth2/__init__.py:40: FutureWarning: You are using a Python version 3.9 past its end of life. Google will upda

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/langchain_google_genai/chat_models.py:47: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  from google.generativeai.caching import CachedContent  # type: ignore[import]


ValidationError: 1 validation error for GoogleGenerativeAIEmbeddings
model
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

## 3. Vector Store (FAISS)

O FAISS (Facebook AI Similarity Search) é uma biblioteca eficiente para busca de similaridade. Vamos indexar nossos chunks.

In [5]:
from langchain_community.vectorstores import FAISS

# Cria o índice vetorial
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# Podemos consultar o índice
retriever = vectorstore.as_retriever(search_kwargs={"k": 2}) # k=2 retorna os 2 mais similares

NameError: name 'embeddings' is not defined

## 4. Busca de Similaridade

Vamos ver se ele encontra trechos relevantes sobre "Tool use".

In [6]:
docs = vectorstore.similarity_search("What are autonomous agents?")

for doc in docs:
    print(f"--- CONTEÚDO (len={len(doc.page_content)}) ---")
    print(doc.page_content[:300] + "...")
    print("\n")

NameError: name 'vectorstore' is not defined

## Conclusão

Criamos nosso "banco de dados" de conhecimento vetorial.

No próximo notebook, vamos juntar tudo: pegar a pergunta do usuário, buscar no FAISS, e passar para o LLM responder (RAG completo).